# Assignment: CERES data

To solve the following exercises you can copy and paste code from the previous notebook. The code modifications required to solve the exercises are minimal (e.g. changing the name of a variable, add a small computation...): don't think too "complicated"!

## Importing the modules

This one is easy. I'll do it for you:

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import numpy as np  # numerical library
import xarray as xr  # netCDF library
import cartopy  # Map projections libary
import cartopy.crs as ccrs  # Projections list
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size

## TOA fluxes 

Read the TOA dataset we used during the lesson. Do you remember all variables it contains?

In [ ]:
ds = xr.open_dataset('../data/CERES_EBAF-TOA_Ed4.1_Clim-2005-2015.nc')
ds

### Albedo 

Compute the the climatological mean of clear-sky planetary albedo $\overline{\alpha_{P_{clr}}}$ and plot it on a map. Analyse the plot.

In [ ]:
alpha_clr = ds.toa_sw_clr_c_clim.mean(dim='month') / ds.solar_clim.mean(dim='month')
ax = plt.axes(projection=ccrs.Robinson())
alpha_clr.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

Repeat the operation with all-sky planetary albedo $\overline{\alpha_{P_{all}}}$. Where are the largest differences? Can you plot the difference between the two on a map, too?

In [ ]:
alpha_all = ds.toa_sw_all_clim.mean(dim='month') / ds.solar_clim.mean(dim='month')
ax = plt.axes(projection=ccrs.Robinson())
alpha_all.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

In [ ]:
alpha_all = ds.toa_sw_all_clim.mean(dim='month') / ds.solar_clim.mean(dim='month')
ax = plt.axes(projection=ccrs.Robinson())
(alpha_all - alpha_clr).plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

Now plot the zonal, climatological means $\left[ \overline{\alpha_{P_{all}}} \right]$ and $\left[ \overline{\alpha_{P_{clr}}} \right]$ on the same plot. Add a legend to it!

In [ ]:
alpha_all.mean(dim='lon').plot(label='$\\left[ \\overline{\\alpha_{P_{all}}} \\right]$')
alpha_clr.mean(dim='lon').plot(label='$\\left[ \\overline{\\alpha_{P_{clr}}} \\right]$')
plt.xlim(-90, 90)
plt.legend();

Compute the global average of $\overline{\alpha_{P_{all}}}$ and $\overline{\alpha_{P_{clr}}}$ (remember to weight according to latitude!). Compare the values you obtain with the ones we mentioned in the lecture.

In [ ]:
weight = np.cos(np.deg2rad(ds.lat))
weight = weight / weight.sum()

In [ ]:
print('Average All Sky Albedo: {:.2f}'.format(float((alpha_all.mean(dim='lon') * weight).sum())))
print('Average Clr Sky Albedo: {:.2f}'.format(float((alpha_clr.mean(dim='lon') * weight).sum())))

### Longwave outgoing radiation 

Repeat the operations above with $LW_{all} $ and  $LW_{clr}$ (i.e.: maps of $\overline{LW_{all}}$, $\overline{LW_{clr}}$, line plots of $\left[ \overline{LW_{all}} \right]$, $\left[ \overline{LW_{clr}} \right]$). What is the global effect of clouds on outgoing longwave radiation?

In [ ]:
# Map 1
lw_clr = ds.toa_lw_clr_c_clim.mean(dim='month')
ax = plt.axes(projection=ccrs.Robinson())
lw_clr.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); ax.set_title('$\overline{LW_{Clr}}$');
# Map 2
plt.figure();
lw_all = ds.toa_lw_all_clim.mean(dim='month')
ax = plt.axes(projection=ccrs.Robinson())
lw_all.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); ax.set_title('$\overline{LW_{All}}$');
# Diff
plt.figure();
ax = plt.axes(projection=ccrs.Robinson())
(lw_all - lw_clr).plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); ax.set_title('$\overline{LW_{All}} - \overline{LW_{Clr}}$');

In [ ]:
ds.toa_lw_all_clim.mean(dim=['month', 'lon']).plot(label='$\\left[ \\overline{LW_{P_{all}}} \\right]$')
ds.toa_lw_clr_c_clim.mean(dim=['month', 'lon']).plot(label='$\\left[ \\overline{LW_{P_{all}}} \\right]$')
plt.xlim(-90, 90)
plt.legend();

## Surface fluxes 

Now open the EBAF-Surface dataset, available for download [here](https://www.dropbox.com/s/r0armbs8ip4op1f/CERES_EBAF-Surface_Ed4.1_Clim-2005-2015.nc?dl=1).

In [ ]:
ds_surf = xr.open_dataset('../data/CERES_EBAF-Surface_Ed4.1_Clim-2005-2015.nc')
ds_surf

### Surface albedo 

Compute the all-sky surface albedo $\overline{\alpha_{S_{all}}}$. Plot it on a map.

In [ ]:
ds_mean = ds_surf.mean(dim='month')  # see what I've done here?
alpha_s =  ds_mean.sfc_sw_up_all_clim / ds_mean.sfc_sw_down_all_clim
# Map
ax = plt.axes(projection=ccrs.Robinson())
alpha_s.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines();

Compute the global average of $\overline{\alpha_{s}}$.

In [ ]:
print('Average All Sky Surface Albedo: {:.2f}'.format(float((alpha_s.mean(dim='lon') * weight).sum())))

### Surface energy balance

Now compute the net surface energy intake $\overline{SEB} =  \overline{SW_{in}} - \overline{SW_{out}} + \overline{LW_{in}} - \overline{LW_{out}}$. Plot it on a map and analyse your results. Where does the surface gain most energy? Is the net radiative energy a gain or a loss for the surface of the globe?

In [ ]:
net = ds_mean.sfc_sw_down_all_clim - ds_mean.sfc_sw_up_all_clim + ds_mean.sfc_lw_down_all_clim - ds_mean.sfc_lw_up_all_clim

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
net.plot(ax=ax, transform=ccrs.PlateCarree()) 
ax.coastlines(); ax.gridlines(); 

In [ ]:
print('Net radiative balance: {:.2f} W m-2'.format(float((net.mean(dim='lon') * weight).sum())))

Compute the global averages of each term and compare them to the values we discussed in the lecture. For reference, here is the figure again:

<img src="http://www.skepticalscience.com/pics/Figure1.png" width="50%"  align="left">

In [ ]:
print('SWdown: {:.2f} W m-2'.format(float((ds_mean.sfc_sw_down_all_clim.mean(dim='lon') * weight).sum())))
print('SWup: {:.2f} W m-2'.format(float((ds_mean.sfc_sw_up_all_clim.mean(dim='lon') * weight).sum())))
print('LWdown: {:.2f} W m-2'.format(float((ds_mean.sfc_lw_down_all_clim.mean(dim='lon') * weight).sum())))
print('LWup: {:.2f} W m-2'.format(float((ds_mean.sfc_lw_up_all_clim.mean(dim='lon') * weight).sum())))

**Discuss the processes that will counterbalance this net radiative energy imbalance, in the oceans and on land!**